In [1]:
import pythoncom
import win32com.client

def main():
    # Initialize COM
    pythoncom.CoInitialize()
    
    # Create or get AutoCAD application
    acad = win32com.client.Dispatch("AutoCAD.Application")
    acad.Visible = True  # Make AutoCAD visible (if you want to see the process)

    # Create a new drawing
    doc = acad.Documents.Add()
    modelspace = doc.ModelSpace

    # Define basic parameters
    floor_height = 3000   # Height of each floor (example in mm)
    bay_width = 5000      # Width of each bay (example in mm)
    
    # We'll assume a rectangular floor plan with 2 bays along the X-axis.
    # So each floor is effectively 2 * bay_width wide.
    # Let's define the depth (Y-axis) of each floor as some arbitrary value.
    floor_depth = 3000
    
    # Coordinates for the first floor outline (2 bays):
    # We'll place the lower left corner at (0, 0).
    # 1st Bay: from X=0 to X=bay_width
    # 2nd Bay: from X=bay_width to X=2*bay_width
    # Depth is from Y=0 to Y=floor_depth
    floor1_coords = [
        (0, 0, 0),
        (2*bay_width, 0, 0),
        (2*bay_width, floor_depth, 0),
        (0, floor_depth, 0),
        (0, 0, 0)  # close the rectangle
    ]
    
    # Coordinates for the second floor outline, offset in Z by floor_height
    floor2_coords = [
        (0, 0, floor_height),
        (2*bay_width, 0, floor_height),
        (2*bay_width, floor_depth, floor_height),
        (0, floor_depth, floor_height),
        (0, 0, floor_height)  # close the rectangle
    ]
    
    # Draw outlines for floor 1
    draw_polyline(modelspace, floor1_coords)
    
    # Draw outlines for floor 2
    draw_polyline(modelspace, floor2_coords)
    
    # Optionally, connect vertical lines (the corners) if you want 3D lines
    # representing the columns at each corner:
    connect_floors(modelspace, floor1_coords, floor2_coords)
    
    print("2 floors and 2 bays have been drawn.")

def draw_polyline(modelspace, coords):
    """
    Draw a closed polyline by connecting the given list of coordinates in sequence.
    coords: list of tuples (x, y, z)
    """
    for i in range(len(coords) - 1):
        start = coords[i]
        end = coords[i+1]
        modelspace.AddLine(start, end)

def connect_floors(modelspace, floor1_coords, floor2_coords):
    """
    Connect corresponding points from floor1 to floor2 as vertical lines
    (like columns).
    """
    num_points = min(len(floor1_coords), len(floor2_coords))
    # floor1_coords and floor2_coords are closed polylines, so the last point repeats the first
    # We'll stop at second to last so as to avoid duplication
    for i in range(num_points - 1):
        p1 = floor1_coords[i]
        p2 = floor2_coords[i]
        modelspace.AddLine(p1, p2)

if __name__ == "__main__":
    main()


com_error: (-2147352567, 'Exception occurred.', (0, None, None, None, 0, -2147024809), None)

In [9]:
import pythoncom
import win32com.client
import time

def main():
    pythoncom.CoInitialize()

    # Start or connect to AutoCAD
    print("Attempting to start or connect to AutoCAD...")
    acad = win32com.client.Dispatch("AutoCAD.Application")
    acad.Visible = True
    print("AutoCAD was started successfully.")

    # 1) Try to get an existing active document
    doc = None
    try:
        doc = acad.ActiveDocument
        if not doc:
            raise RuntimeError("No active document.")
        print(f"Using ActiveDocument: {doc.Name}")
    except Exception as e:
        print("No active document found or error accessing it:", e)

    # 2) If no active doc is found, attempt to create a new one
    if doc is None:
        try:
            print("Attempting to create a new drawing from 'acad.dwt'...")
            doc = acad.Documents.Add("acad.dwt")
            print("New document created.")
        except Exception as e:
            print("Failed to create a new document:", e)
            doc = None

    if doc is None:
        raise RuntimeError(
            "Could not get or create a valid AutoCAD document. "
            "Verify you have a full AutoCAD version and a valid template."
        )

    # Allow AutoCAD a moment to finish setting up the new document
    time.sleep(2)

    # Circle parameters: center and radius
    center_point = (100.0, 100.0, 0.0)
    radius       = 50.0

    print(f"Attempting to draw a circle with center {center_point} and radius {radius}...")

    # 3) Try doc.ModelSpace.AddCircle
    try:
        ms = doc.ModelSpace
        ms.AddCircle(center_point, radius)
        print("SUCCESS: Circle drawn via doc.ModelSpace.AddCircle(...)")
    except Exception as e:
        print(f"FAILED via doc.ModelSpace.AddCircle(...): {e}")

        # 4) Fallback: doc.Blocks("Model")
        try:
            model_block = doc.Blocks("Model")
            model_block.AddCircle(center_point, radius)
            print("SUCCESS: Circle drawn via doc.Blocks('Model').AddCircle(...)")
        except Exception as e2:
            print(f"FAILED via doc.Blocks('Model').AddCircle(...): {e2}")
            print("No further fallback methods available.")
            return  # Stop here

    # Zoom extents so we can see the geometry
    try:
        doc.SendCommand("._zoom _extents ")
        print("Zoom Extents command sent.")
    except Exception as e:
        print("Failed to send Zoom Extents command:", e)

    print("Script completed. Check AutoCAD to see if the circle was drawn.")

if __name__ == "__main__":
    main()


Attempting to start or connect to AutoCAD...
AutoCAD was started successfully.
Using ActiveDocument: Drawing1.dwg
Attempting to draw a circle with center (100.0, 100.0, 0.0) and radius 50.0...
FAILED via doc.ModelSpace.AddCircle(...): (-2147352567, 'Exception occurred.', (0, None, None, None, 0, -2147024809), None)
FAILED via doc.Blocks('Model').AddCircle(...): (-2147352567, 'Exception occurred.', (0, 'AutoCAD.Application', 'Key not found', 'C:\\Program Files\\Autodesk\\AutoCAD 2024\\HELP\\OLE_ERR.CHM', -2145386476, -2145386476), None)
No further fallback methods available.
